1. **Crear un programa en OpenCV que a) tome una imagen como entrada, la cargue en tonos de gris, la visualice, muestre su histograma, y b) iguale su histograma y muestre en otra ventana diferente el resultado de esta igualación. El resultado de la operación de igualación se puede ver en las figuras 1 y 2**

![igualación](figura1.jpg)
Figura 1. Ejemplo de igualación

![igualaciónHistograma](figura2.jpg)
Figura 2. Histograma igualado

* **a) Cargamos la imagen de entrada en escala de gris, la visualizamos y mostramos su histograma:**

In [6]:
import numpy as np
import cv2, sys


def calculaHistograma(imagen):
    #calculamos las dimensiones
    dimensiones=imagen.shape
    alto = dimensiones[0]
    ancho = dimensiones[1]
    #calculamos el array con el nÃºmero de pÃ­xeles que possen cada uno de los 256 tonos posibles
    vectorHistograma = np.zeros(256, np.uint32)
    for i in range(alto):
        for j in range(ancho):
            vectorHistograma[imagen[i][j]] += 1
    return vectorHistograma

def creaImagenHistograma(alto):
    anchoHistograma = 256 * 4
    imagenHistograma = np.zeros((alto, anchoHistograma), np.uint8)
    imagenHistograma.fill(255)
    return imagenHistograma

def dibujaHistograma(imagenHistograma, vectorHistograma, altoImagen,nombre):
    valorMaximo = np.max(vectorHistograma)
    vectorHistogramaNormalizado = np.zeros(256, np.uint32)
    posicionXInicialRectangulo = 0
    for i in range(vectorHistograma.size):
        vectorHistogramaNormalizado[i] = (vectorHistograma[i]/valorMaximo) * altoImagen
        #Nota: hay que darse cuenta que la posiciÃ³n (0,0) es la esquina superior izquierda es la esquina superior.  
        cv2.rectangle(imagenHistograma, (posicionXInicialRectangulo, altoImagen - vectorHistogramaNormalizado[i]), (posicionXInicialRectangulo + 4, altoImagen), (0, 0, 0), -1)
        posicionXInicialRectangulo +=4
    cv2.imshow(nombre, imagenHistograma)


nombreImagen ="p4.png"

#cargamos la imagen
imagen = cv2.imread(nombreImagen, cv2.IMREAD_GRAYSCALE)

if (imagen is None):
    print("Error al cargar la imagen")
    sys.exit()
    
#calculamos las dimensiones
dimensiones=imagen.shape
alto = dimensiones[0]
ancho = dimensiones[1]

vectorHistograma = calculaHistograma(imagen)

cv2.imshow("Imagen", imagen)
imagenHistograma = creaImagenHistograma(alto)
dibujaHistograma(imagenHistograma, vectorHistograma, alto,"Histograma")

cv2.waitKey(0)
cv2.destroyAllWindows()


* **iguale su histograma y muestre en otra ventana diferente el resultado de esta igualación (Nota: la explicación del algoritmo de igualación la podemos encontrar en la transparencia 28 del tema 2)**

In [11]:
# Inicializamos MN, l y vectores necesarios
MN = alto * ancho
L = len(vectorHistograma) - 1 
vectorProbabilidades = np.zeros(256, np.float32)
vectorHistogramaFinal = np.zeros(256, np.float32)

# Aplicamos la formula
for i in range(len(vectorHistograma)):
    vectorProbabilidades[i] = vectorHistograma[i]/MN
    s = 0
    for j in range(i+1):
        s += L * vectorProbabilidades[j]
        
    vectorHistogramaFinal[i] = round(s)    
    
# Modifico la imagen original 
for i in range(alto):
    for j in range(ancho):
        imagen[i][j] = vectorHistogramaFinal[imagen[i][j]]

# Mostramos la imagen y su histograma        
cv2.imshow("ImagenFinal", imagen)
imagenHistograma = creaImagenHistograma(alto)
vectorHistogramaNuevo = calculaHistograma(imagen)
dibujaHistograma(imagenHistograma, vectorHistogramaNuevo, alto,"HistogramaFinal")    

cv2.waitKey(0)
cv2.destroyAllWindows()

_Explicación del código:_

La formula esta implmentada de forma que aprovecha en un bucle: 

calculo de probabilidad (pr), sumatoria y redondeo (s)

